# Aiida

https://github.com/BAMresearch/NFDI4IngScientificWorkflowRequirements

## Define workflow with aiida

In [ ]:
import os

In [ ]:
from python_workflow_definition.aiida import write_workflow_json

from aiida_workgraph import WorkGraph, task
from aiida import orm, load_profile
load_profile()

In [ ]:
from workflow import (
    generate_mesh,
    convert_to_xdmf as _convert_to_xdmf,
    poisson as _poisson,
    plot_over_line,
    substitute_macros,
    compile_paper,
)

In [ ]:
convert_to_xdmf = task(outputs=["xdmf_file", "h5_file"])(_convert_to_xdmf)
poisson = task(outputs=["numdofs", "pvd_file", "vtu_file"])(_poisson)

In [ ]:
source_directory = orm.Str(os.path.abspath(os.path.join(os.curdir, "source")))

In [ ]:
domain_size = orm.Float(2.0)

In [ ]:
wg = WorkGraph("wg-nfdi")

In [ ]:
gmsh_output_file = wg.add_task(
    generate_mesh,
    domain_size=domain_size,
    source_directory=source_directory,
)

In [ ]:
meshio_output_dict = wg.add_task(
    convert_to_xdmf,
    gmsh_output_file=gmsh_output_file.outputs.result,
)

In [ ]:
poisson_dict = wg.add_task(
    poisson,
    meshio_output_xdmf=meshio_output_dict.outputs.xdmf_file, 
    meshio_output_h5=meshio_output_dict.outputs.h5_file,
    source_directory=source_directory,
)

In [ ]:
pvbatch_output_file = wg.add_task(
    plot_over_line,
    poisson_output_pvd_file=poisson_dict.outputs.pvd_file, 
    poisson_output_vtu_file=poisson_dict.outputs.vtu_file,
    source_directory=source_directory,
)

In [ ]:
macros_tex_file = wg.add_task(
    substitute_macros,
    pvbatch_output_file=pvbatch_output_file.outputs.result, 
    ndofs=poisson_dict.outputs.numdofs, 
    domain_size=domain_size,
    source_directory=source_directory,
)

In [ ]:
paper_output = wg.add_task(
    compile_paper,
    macros_tex=macros_tex_file.outputs.result, 
    plot_file=pvbatch_output_file.outputs.result,
    source_directory=source_directory,
)

In [ ]:
wg

In [ ]:
workflow_json_filename = "aiida_nfdi.json"

In [ ]:
write_workflow_json(wg=wg, file_name=workflow_json_filename)

In [ ]:
!cat {workflow_json_filename}

## Load Workflow with jobflow

In [ ]:
from python_workflow_definition.jobflow import load_workflow_json

In [ ]:
from jobflow.managers.local import run_locally

In [ ]:
flow = load_workflow_json(file_name=workflow_json_filename)

In [ ]:
result = run_locally(flow)
result

## Load Workflow with pyiron_base

In [ ]:
from python_workflow_definition.pyiron_base import load_workflow_json

In [ ]:
delayed_object_lst = load_workflow_json(file_name=workflow_json_filename)
delayed_object_lst[-1].draw()

In [ ]:
delayed_object_lst[-1].pull()

## Load Workflow with pyiron_workflow

In [ ]:
from python_workflow_definition.pyiron_workflow import load_workflow_json

In [ ]:
wf = load_workflow_json(file_name=workflow_json_filename)

In [ ]:
wf.draw(size=(10,10))

In [ ]:
wf.run()